## Лабораторная работа 4

In [1]:
import scipy.stats as stat
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

### I. Проверка статистических гипотез.

In [2]:
def in_range(x, interval):
    """
    Функция оценивает попадание в диапазон.
        
    @param:
    @return:
    """
    a, b = interval
    if a <= x <= b:
        return True
    return False

In [3]:
def group_data(data, k):
    """
    Функция по группированию данных.
        
    @param:
    @return:
    """
    # переведем ряд в вариационный и вычислим начальную и конечную точку
    x0 = math.floor(min(data))
    xn = math.ceil(max(data))
    
    # найдем границы отрезков
    interval_size = (xn - x0) / k
    bounds = [x0 + i * interval_size for i in range(k + 1)]
    
    # выделим отрезки
    a, b = itertools.tee(bounds)
    next(b, None)
    intervals = list(zip(a, b))
    
    # найдем серидины интервалов
    intervals_mids = [(a + b) / 2 for (a, b) in intervals]
    
    grouped = np.zeros(np.size(data))
    
    for i in range(np.size(data)):
        # находим индекс интервала в который попадает точка исходной выборки
        entry_group = [j for j in range(k) if in_range(data[i], intervals[j])][0]
        # группируем, записывая в финальный массив значение серидины интервала в который попала текущая точка
        grouped[i] = intervals_mids[entry_group]
        
    return grouped

In [4]:
def empirical_cdf(t,data):
    """
    Кумулятивная функция распределения СВ t, или просто функция распределения t, 
    это вероятность того, что x примет значение меньшее или равное t
    
    @param:
    @return:
    """
    return sum([1 for x in data if x <= t]) / np.size(data)

In [5]:
def print_result(statistic, T1, T2 = None):
    """
    Печать результатов.
    
    1. В качестве статистики берем t со шляпкой, и далее строим критическую область.
    2. Далее смотрим на попадание статистики в критическое множество.
    3. Делаем выводы, H0 принимается, или отвергается в пользу H1.
    """
    print(f'статистика:            [{statistic}]')
    
    if T2 is None:
        
        print(f'критическое множество: [-{T1}, {T1}]')
        if -T1 <= statistic <= T1:
            print(f'\nH0 принимается')
        else:
            print(f'\nH0 отвергается в пользу H1')
    else:
        
        print(f'критическое множество: [{T1}, {T2}]')
        if T1 <= statistic <= T2:
            print(f'\nH0 принимается')
        else:
            print(f'\nH0 отвергается в пользу H1')

In [6]:
alpha = 0.05
k = 8
u_half_alpha = -stat.norm.ppf(alpha / 2)

#### Гипотеза №1.

Проверка гипотезы о равенстве мат. ожиданий для разных выборок гаусса.

    a.Группированная выборка.

In [7]:
n_x_gauss = 200
n_y_gauss = 300

x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)
y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)

s2 = ((n_x_gauss - 1) * np.var(x_gauss) + (n_y_gauss-2) * np.var(y_gauss)) / (n_x_gauss + n_y_gauss - 2)
t = (np.mean(x_gauss) - np.mean(y_gauss)) / (s2**0.5 * (1/n_x_gauss + 1/n_y_gauss)**0.5)
t_half_alpha = stat.t.ppf(1-alpha/2, df=n_x_gauss+n_y_gauss-2)

print_result(t, t_half_alpha)

статистика:            [0.26728157187301727]
критическое множество: [-1.9647389829672648, 1.9647389829672648]

H0 принимается


    a.Негруппированная выборка.

In [8]:
x_group = group_data(y_gauss, k)
y_group = group_data(x_gauss, k)

s2 = ((n_x_gauss - 1) * np.var(x_group) + (n_y_gauss-2) * np.var(y_group)) / (n_x_gauss + n_y_gauss - 2)
t = (np.mean(x_group) - np.mean(y_group)) / (s2**0.5 * (1/n_x_gauss + 1/n_y_gauss)**0.5)
t_half_alpha = stat.t.ppf(1-alpha/2, df=n_x_gauss+n_y_gauss-2)

print_result(t, t_half_alpha)

статистика:            [-0.41029328925649816]
критическое множество: [-1.9647389829672648, 1.9647389829672648]

H0 принимается


#### Гипотеза №2.

Проверка равенства нулю среденего значения tn для выборки с шумом для нормального распределения.

In [9]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
y = 5 * x_gauss + 10 * np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
z = y - x_gauss

t = np.mean(z) / (np.std(z) / n_x_gauss**0.5)
t_half_alpha = stat.t.ppf(1-alpha/2, df=n_x_gauss)

print_result(t, t_half_alpha)

статистика:            [5.513402796938266]
критическое множество: [-1.9718962236316089, 1.9718962236316089]

H0 отвергается в пользу H1


#### Гипотеза №3.

Проверяем гипотезу,что в левой и правой половине одинаковый параметр после деления выборки с распределением пуассона пополам.

In [10]:
lambda_x = 9
n_x_poisson = 200

x_poisson = stat.poisson.rvs(lambda_x, size=n_x_poisson)

x1_poisson = x_poisson[:n_x_poisson//2]
x2_poisson = x_poisson[n_x_poisson//2:]

t = (np.mean(x1_poisson) - np.mean(x2_poisson)) / (np.mean(x1_poisson) + np.mean(x2_poisson))**0.5

t_left_quant = stat.norm.ppf(alpha/2)
t_right_quant = stat.norm.ppf(1 - alpha/2)

print_result(t, t_left_quant, t_right_quant)

статистика:            [0.09379228369755709]
критическое множество: [-1.9599639845400545, 1.959963984540054]

H0 принимается


#### Гипотеза №4.

Проверка гипотезы о равенстве дисперсий по критерию фишера для выборки из гипотезы №1.

In [11]:
n_x_gauss = 200
n_y_gauss = 300

x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)
y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)

fisher_left_quan = stat.f.ppf(alpha / 2, dfn = n_x_gauss-1, dfd = n_y_gauss - 1)
fisher_right_quan = stat.f.ppf(1 - alpha / 2, dfn = n_x_gauss-1, dfd = n_y_gauss - 1)

F = np.var(x_gauss) / np.var(y_gauss)

print_result(F, fisher_left_quan, fisher_right_quan)

статистика:            [0.6727938041944701]
критическое множество: [0.7729893672926245, 1.285204934412516]

H0 отвергается в пользу H1


#### Гипотеза №5.

Проверка гипотезы о равенстве дисперсий в правой и левой части с помощью фишера после разбиения на два части выборки из гипотезы №1. 

In [12]:
n_y_gauss = 300

y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)

y1_gauss = y_gauss[:n_y_gauss // 2]
y2_gauss = y_gauss[n_y_gauss // 2:]

fisher_left_quan = stat.f.ppf(alpha / 2, dfn = n_y_gauss / 2 - 1, dfd = n_y_gauss / 2 - 1)
fisher_right_quan = stat.f.ppf(1 - alpha / 2, dfn = n_y_gauss / 2 - 1, dfd = n_y_gauss / 2 - 1)

F = np.var(y1_gauss) / np.var(y2_gauss)

print_result(F, fisher_left_quan, fisher_right_quan)

статистика:            [0.8234273677840515]
критическое множество: [0.7244337450679404, 1.3803884852247126]

H0 принимается


#### Гипотеза №6.

Проверка гипотезы о коррелированности двух выборок гаусса для данных из гипотезы №2 через коэффициент пирсона.

In [13]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
y = 5 * x_gauss + np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
ro, _ = stat.pearsonr(x_gauss, y)

t = ro / (1 - ro**2)**0.5 * (n_x_gauss - 2)**0.5
t_right_quant = stat.t.ppf(1-alpha/2, df=n_x_gauss)
t_left_quant = stat.t.ppf(alpha/2, df=n_x_gauss)

print_result(t, t_left_quant, t_right_quant)

статистика:            [71.4086277813903]
критическое множество: [-1.9718962236316093, 1.9718962236316089]

H0 отвергается в пользу H1


#### Гипотеза №7.

Проверка гипотезы о некоррелированности двух частей выборки из гипотезы №6 при делении ее пополам через коэффициент пиросона.

In [14]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
x1_gauss = x_gauss[:n_x_gauss//2]
x2_gauss = x_gauss[n_x_gauss//2:]

ro, pp = stat.pearsonr(x1_gauss, x2_gauss)

t = ro / (1 - ro**2)**0.5 * (n_x_gauss//2 - 2)**0.5
t_right_quant = stat.t.ppf(1-alpha/2, df=n_x_gauss//2)
t_left_quant = stat.t.ppf(alpha/2, df=n_x_gauss//2)

print_result(t, t_left_quant, t_right_quant)

статистика:            [1.3165825299616025]
критическое множество: [-1.983971518449634, 1.9839715184496334]

H0 принимается


### II. Проверка критериев согласия.

In [15]:
def group_data(data, k):
    
    x0 = math.floor(min(data))
    xn = math.ceil(max(data))
    
    bounds = [x0 + i * (xn - x0) / k for i in range(k + 1)]
    
    # выделим отрезки
    a, b = itertools.tee(bounds)
    next(b, None)
    intervals = list(zip(a, b))
    
    invervals_mids = [(a + b) / 2 for (a, b) in intervals]
    grouped = np.zeros(np.size(invervals_mids))
    
    for i in range(np.size(data)):
        entry_group = [j for j in range(k) if in_range(data[i], intervals[j])][0]
        grouped[entry_group] += 1
        
    return intervals, grouped

In [25]:
def chi2_normality_test(data):
    mean = np.mean(data)
    std = np.std(data)
    
    intervals, freq = group_data(data, k=10)
    exp = [np.size(data)*(stat.norm.cdf(b, 
                                        loc=mean, 
                                        scale=std)-stat.norm.cdf(a, loc=mean, scale=std)) for (a,b) in intervals]
    
    chi2 = np.sum((freq - exp)**2 / exp)
    chi2_quant = stat.chi2.ppf(1 - alpha, df=k-2) # -2 for 2 estimated parameters (location and scale)

    print(f'статистика:            [{chi2}]')
    print(f'критическое значение:  [{chi2_quant}]')
    
    if chi2 < chi2_quant:
        print(f'\nH0 принимается')
    else:
        print(f'\nH0 отвергается в пользу H1')

In [26]:
def kolmogorov_normality_test(data):
    mean = np.mean(data)
    std = np.std(data)
    
    dif = [np.abs(empirical_cdf(t,data) - stat.norm.cdf(t,loc=mean,scale=std)) for t in data]
    
    k_statistic = np.size(data)**0.5 * np.max(dif)
    kolmogorov_quant = stat.kstwobign.ppf(1-alpha)
    
    print(f'статистика:            [{k_statistic}]')
    print(f'критическое значение:  [{kolmogorov_quant}]')
    
    if k_statistic < kolmogorov_quant:
        print(f'\nH0 принимается')
    else:
        print(f'\nH0 отвергается в пользу H1')

In [39]:
def KS_test(x, y):
    n1 = np.size(x)
    n2 = np.size(y)
    
    cumulated = np.append(x,y)
    dif = [np.abs(empirical_cdf(t,x) - empirical_cdf(t,y)) for t in cumulated]
    
    k_statistic = np.max(dif)
    crit_value = ((n1 + n2) / (n1 * n2))**0.5 * (-np.log(alpha/2) * 0.5)**0.5
    
    print(f'статистика:            [{k_statistic}]')
    print(f'критическое значение:  [{crit_value}]')
    
    if crit_value > k_statistic:
        print(f'\nH0 принимается')
    else:
        print(f'\nH0 отвергается в пользу H1')

In [40]:
n_x_gauss = 400

#### 1.Критерий Хи-квадрат.

Проверка гипотезы нормальности для всей выборки.

    a.Нормальное распределение

In [41]:
x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)
chi2_normality_test(x_gauss)

статистика:            [4.337680980340313]
критическое значение:  [12.591587243743977]

H0 принимается


    b.Нормальное распределение + noise 1

In [42]:
x_gauss_uni_noise = x_gauss + np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
chi2_normality_test(x_gauss_uni_noise)

статистика:            [11.298706236772823]
критическое значение:  [12.591587243743977]

H0 принимается


    c.Нормальное распределение + noise 2

In [43]:
x_gauss_cauchy_noise = x_gauss + 0.01 * stat.cauchy.rvs(loc=0, scale=1, size=n_x_gauss)
chi2_normality_test(x_gauss_cauchy_noise)

статистика:            [8.085152558468067]
критическое значение:  [12.591587243743977]

H0 принимается


#### 2. Критерий Колмогорова.

Проверка гипотезы нормальности для всей выборки.

    a.Нормальное распределение

In [44]:
kolmogorov_normality_test(x_gauss)

статистика:            [0.4789132112237371]
критическое значение:  [1.3580986393225505]

H0 принимается


    b.Нормальное распределение + noise 1

In [45]:
kolmogorov_normality_test(x_gauss_uni_noise)

статистика:            [0.9051556561893259]
критическое значение:  [1.3580986393225505]

H0 принимается


    c.Нормальное распределение + noise 2

In [46]:
kolmogorov_normality_test(x_gauss_cauchy_noise)

статистика:            [0.46742642677427115]
критическое значение:  [1.3580986393225505]

H0 принимается


#### 3. Критерий Колмогорова-Смирнова.

    a.Нормальное распределение

In [47]:
x1_gauss = x_gauss[:n_x_gauss // 2]
x2_gauss = x_gauss[n_x_gauss // 2:]
KS_test(x1_gauss, x2_gauss)

статистика:            [0.08999999999999997]
критическое значение:  [0.13581015157406195]

H0 принимается


    b.Нормальное распределение + noise 1

In [48]:
x1_uni_noise = x_gauss_uni_noise[:n_x_gauss // 2]
x2_uni_noise = x_gauss_uni_noise[n_x_gauss // 2:]
KS_test(x1_uni_noise, x2_uni_noise)

статистика:            [0.08999999999999997]
критическое значение:  [0.13581015157406195]

H0 принимается


    c.Нормальное распределение + noise 2

In [49]:
x1_cauchy_noise = x_gauss_cauchy_noise[:n_x_gauss // 2]
x2_cauchy_noise = x_gauss_cauchy_noise[n_x_gauss // 2:]
KS_test(x1_cauchy_noise, x2_cauchy_noise)

статистика:            [0.08499999999999996]
критическое значение:  [0.13581015157406195]

H0 принимается
